## Web References

### System

- [How to copy a file without using scp inside an ssh session?](https://superuser.com/questions/291423/how-to-copy-a-file-without-using-scp-inside-an-ssh-session)

### PySpark

- [Complete Machine Learning Project with PySpark MLlib Tutorial](https://www.youtube.com/watch?v=1a7bB1ZcZ3k)
- [The ONLY PySpark Tutorial You Will Ever Need.](https://www.youtube.com/watch?v=cZS5xYYIPzk)
- [PySpark When Otherwise | SQL Case When Usage](https://sparkbyexamples.com/pyspark/pyspark-when-otherwise/)
- [Spark rlike() Working with Regex Matching Examples](https://sparkbyexamples.com/spark/spark-rlike-regex-matching-examples/)
- [spark dataframe drop duplicates and keep first](https://stackoverflow.com/questions/38687212/spark-dataframe-drop-duplicates-and-keep-first)

### Anomaly Detection

- [How to Build an Anomaly Detection Engine with Spark, Akka and Cassandra](https://learning.oreilly.com/videos/how-to-build/9781491955253/9781491955253-video244545/)
- [Real Time Detection of Anomalies in the Database Infrastructure using Apache Spark](https://www.youtube.com/watch?v=1IsMMmug5q0)

### Other

- [What is CRISP DM?](https://www.datascience-pm.com/crisp-dm-2/)

### Internet Traffic

- [Data mining approach for predicting the daily Internet data traffic of a smart university](https://journalofbigdata.springeropen.com/articles/10.1186/s40537-019-0176-5)

## Import Libraries

In [69]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, asc, desc, col

## HDFS Preparation

In [70]:
%%bash
#!/bin/bash

# delete the 
if hadoop fs -test -d router; then
    # delete the output directory
    hadoop fs -rm -r router/output

    # create a new output directory
    hadoop fs -mkdir router/output
else
    # create the router directory and upload the input files
    hadoop fs -mkdir router
    hadoop fs -mkdir router/sample
    hadoop fs -put data/bandwidth.csv router/sample/

    hadoop fs -put data/raw router/raw

    # create the output directory
    hadoop fs -mkdir router/output
fi

hadoop fs -ls router/raw

Deleted router/output
Found 72 items
-rw-r--r--   3 jfoul001 users     150806 2022-03-05 10:07 router/raw/20220304T2210.csv.gz
-rw-r--r--   3 jfoul001 users     149945 2022-03-05 10:07 router/raw/20220304T2220.csv.gz
-rw-r--r--   3 jfoul001 users     149142 2022-03-05 10:07 router/raw/20220304T2230.csv.gz
-rw-r--r--   3 jfoul001 users     151781 2022-03-05 10:07 router/raw/20220304T2240.csv.gz
-rw-r--r--   3 jfoul001 users     150343 2022-03-05 10:07 router/raw/20220304T2250.csv.gz
-rw-r--r--   3 jfoul001 users     149798 2022-03-05 10:07 router/raw/20220304T2300.csv.gz
-rw-r--r--   3 jfoul001 users     148781 2022-03-05 10:07 router/raw/20220304T2310.csv.gz
-rw-r--r--   3 jfoul001 users     148639 2022-03-05 10:07 router/raw/20220304T2320.csv.gz
-rw-r--r--   3 jfoul001 users     149012 2022-03-05 10:07 router/raw/20220304T2330.csv.gz
-rw-r--r--   3 jfoul001 users     150019 2022-03-05 10:07 router/raw/20220304T2340.csv.gz
-rw-r--r--   3 jfoul001 users     150122 2022-03-05 10:07 route

## Initialize the Spark Session

In [71]:
spark = SparkSession.builder.appName('cw02').getOrCreate()
spark

## I. Data Understanding

Identify, collect, and analyze the data sets that will help accomplish the project goals

### A. Collect Initial Data

In [103]:
input_path = 'router/sample/bandwidth.csv'
#input_path = 'router/raw/'

# read the full batch to process
df_batch = spark.read.csv(input_path, header=False, inferSchema=True) \
    .toDF('Direction','Interval Length','Intervals Saved','IP','Interval Start','Interval End','Bytes Used')

In [104]:
# drop the intervals saved column
df_batch = df_batch.drop('Intervals Saved')
df_batch.count()

20278

Remove records related to a specific upload or download class. This is done because this information is not relevant to the project, and have an additional column that will complicate data processing.

In [105]:
df_batch_preproc = df_batch.where(df_batch['Interval Length'].contains('class_') == False)
df_batch_preproc.count()

17256

### B. Describe data

Examine the data and document its surface properties like data format, number of records, or field identities.

#### 1. Data Format

In [106]:
df_batch_preproc.printSchema()

root
 |-- Direction: string (nullable = true)
 |-- Interval Length: string (nullable = true)
 |-- IP: string (nullable = true)
 |-- Interval Start: string (nullable = true)
 |-- Interval End: integer (nullable = true)
 |-- Bytes Used: long (nullable = true)



In [107]:
df_batch_preproc.count()

17256

#### 2. Remove Duplicates

Many duplicate records are present in the raw data since the data snapshots overlaps.

In [108]:
# drop duplicates
df_unique = df_batch_preproc.drop_duplicates()
df_unique.count()

16272

There are duplicates where only the `Bytes Used` field is different due to when the data snapshot was taken.

In [110]:
df_unique = df_unique.groupBy(['Direction','Interval Length','IP','Interval Start','Interval End']) \
    .max('Bytes Used') \
    .withColumnRenamed('max(Bytes Used)', 'Bytes Used')

df_unique.count()

16264

In [111]:
df_unique.show(5)

+---------+---------------+-------------+--------------+------------+----------+
|Direction|Interval Length|           IP|Interval Start|Interval End|Bytes Used|
+---------+---------------+-------------+--------------+------------+----------+
| download|          month|192.168.3.218|    1622498400|  1625090400| 326916468|
| download|              2|     COMBINED|    1646314332|  1646314334|    698185|
|   upload|            day|192.168.3.104|    1644962400|  1645048800|       450|
|   upload|            day|192.168.3.117|    1644444000|  1644530400|         0|
|   upload|              2|192.168.1.107|    1646314256|  1646314258|         0|
+---------+---------------+-------------+--------------+------------+----------+
only showing top 5 rows



### C. Explore data

Dig deeper into the data. Query it, visualize it, and identify relationships among the data.

#### 1. Unique Categories

In [112]:
# if an IP address is stored in the IP column recode it as simply 'IP'
df_interval_type = df_unique.withColumn('interval_type', 
    when(df_unique['IP']
    .rlike('^((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)$'), 'IP')
    .otherwise(df_unique['IP'])
)

df_interval_type.groupBy(['interval_type']).count().show()

+-------------+-----+
|interval_type|count|
+-------------+-----+
|           IP|12106|
|     COMBINED| 4158|
+-------------+-----+



In [114]:
# record count per catergory
df_interval_type.groupBy(['Interval Length', 'interval_type']) \
    .count() \
    .orderBy(['Interval Length', 'interval_type',], ascending=True) \
    .show(df_interval_type.count())

+---------------+-------------+-----+
|Interval Length|interval_type|count|
+---------------+-------------+-----+
|            180|     COMBINED|  960|
|              2|     COMBINED|  904|
|              2|           IP| 2700|
|           7200|     COMBINED|  720|
|            900|     COMBINED|   50|
|            900|           IP| 1165|
|            day|     COMBINED|  732|
|            day|           IP| 3363|
|           hour|     COMBINED|   50|
|           hour|           IP| 1271|
|         minute|     COMBINED|  720|
|         minute|           IP|  722|
|          month|     COMBINED|   22|
|          month|           IP| 2885|
+---------------+-------------+-----+



#### 2. How many intervals are available for the various categories?

This number should be less as for every interval there could be multiple IP addresses (where `interval_type == 'IP'`).

In [115]:
df_interval_type.drop_duplicates(['Direction', 'Interval Length', 'interval_type', 'Interval Start', 'Interval End']) \
    .groupBy(['Interval Length', 'interval_type']) \
    .count() \
    .orderBy(['Interval Length', 'interval_type',], ascending=True) \
    .show(df_interval_type.count())

+---------------+-------------+-----+
|Interval Length|interval_type|count|
+---------------+-------------+-----+
|            180|     COMBINED|  960|
|              2|     COMBINED|  904|
|              2|           IP|  900|
|           7200|     COMBINED|  720|
|            900|     COMBINED|   50|
|            900|           IP|   50|
|            day|     COMBINED|  732|
|            day|           IP|   64|
|           hour|     COMBINED|   50|
|           hour|           IP|   50|
|         minute|     COMBINED|  720|
|         minute|           IP|   32|
|          month|     COMBINED|   22|
|          month|           IP|   28|
+---------------+-------------+-----+



#### 3. Average IP addresses for each `Interval Length`